#NanoNet
NanoNet is a novel deep learning-based end-to-end modeling tool that given a 
sequence directly produces the 3D coordinates of the Cβ atoms of the entire VH domain.
It can be used in order to predict structures of nanobodies, VH regions of mAbs and VB regions of TCRs.

The source code and the trained model can be found [here](https://github.com/deepmind/alphafold)

**Note**: If you are predicting many structures in a single run, it is recomanded that you use the option 'write_into_single_pdb_file'. this will write all the predicted structures into a single PDB file and reduce running time and space.


<strong>for Citation use: </strong> [NanoNet paper](https://www.biorxiv.org/content/10.1101/2021.08.03.454917v1) (preprint)


In [ ]:
#@title Input Nb sequence/Nb fasta file (creates a Nb structure for each entry in the fasta file)
from google.colab import files
import re
import os

protein = 'TCR VB' #@param ["Nb/mAb heavy chain", "TCR VB"]
input_type = 'Sequence (String)' #@param ["Sequence (String)", "Fasta file"]
input_nb = 'QLVETGGGLVQSGGSLRLSCAASGFTLDNYNIGWFRQAPGKEYGGVSCISSSDGSTYYADSVKGRFTISRDNAKNTVYLQMNNLKPEDTDVYYCAATKYGSSCPIRPYDYWGQGTQVT' #@param {type:"string"}
output_dir = 'NanoNetResults' #@param {type:"string"}

# remove whitespaces
output_dir = "".join(output_dir.split())
output_dir = re.sub(r'\W+', '', output_dir)


# get NanoNet trained model path
tcr_modeling = False if protein == 'Nb/mAb heavy chain' else True

#@markdown ---
#@markdown ### Advanced settings
write_into_single_pdb_file = True #@param {type:"boolean"}
# reconsrtuct_side_chains_using_modeller = True #@param {type:"boolean"}
# reconsrtuct_side_chains_using_scwrl = "" #@param {type:"string"}
##@markdown  (insert Scwrl4 executable path)
reconsrtuct_side_chains_using_modeller = False
reconsrtuct_side_chains_using_scwrl = False

#@markdown ---
#@markdown ### Saving options
save_to_google_drive = False #@param {type:"boolean"}
#@markdown ---
#@markdown  to run NanoNet hit `Runtime` -> `Run all`

if input_type == 'Sequence (String)':  
  # remove whitespaces
  input_nb = "".join(input_nb.split())
  input_nb = re.sub(r'[^a-zA-Z]','', input_nb).upper()
  with open("nb_fasta.fa", "w") as fa_file:
    fa_file.write("> model0\n")
    fa_file.write("{}\n".format(input_nb))
  input_nb = "nb_fasta.fa"

if not os.path.exists(input_nb):
  print("can't find fasta file {}.".format(input_nb))

if save_to_google_drive == True:
  from pydrive.drive import GoogleDrive
  from pydrive.auth import GoogleAuth
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("Saving results into Drive")

In [ ]:
#@title Clone NanoNet trained model
%%bash

if [ ! -f NanoNetReady ]; then
  # install dependencies
  pip -q install biopython

  # download model
  if [ ! -d "NanoNet/" ]; then
    git clone https://github.com/dina-lab3D/NanoNet --quiet
  fi
  touch NanoNetReady
fi


In [ ]:
#@title Predict Nb structure

#@title Import libraries
os.chdir("/content/")
if "NanoNet" not in dir():
  from timeit import default_timer as timer
  import sys
  sys.path.insert(0, '/content/NanoNet/')
  from NanoNet import *

nanonet_dir_path = "/content/NanoNet"
nano_net_path = os.path.join(nanonet_dir_path, 'NanoNetTCR') if tcr_modeling else os.path.join(nanonet_dir_path, 'NanoNet')
error = False

if not os.path.exists(nano_net_path):
    print("Can't find trained NanoNet '{}', aborting.".format(nano_net_path), file=sys.stderr)
    error = True
if reconsrtuct_side_chains_using_scwrl and not os.path.exists(reconsrtuct_side_chains_using_scwrl):
    print("Can't find Scwrl4 '{}', aborting.".format(reconsrtuct_side_chains_using_scwrl), file=sys.stderr)
    error = True
if write_into_single_pdb_file and (reconsrtuct_side_chains_using_modeller or reconsrtuct_side_chains_using_scwrl):
    print("Can't reconstruct side chains with single_file option.",file=sys.stderr)
    error = True

if not error:
    start = timer()
    run_nanonet(input_nb, nano_net_path, write_into_single_pdb_file, output_dir, reconsrtuct_side_chains_using_modeller, reconsrtuct_side_chains_using_scwrl)
    end = start = timer()
    os.chdir("/content/")
    print("NanoNet ended succsufully, total time: {}".format(end-start))


In [ ]:
#@title Download results


!zip -FSr $output_dir".zip" $output_dir
files.download(f"{output_dir}.zip")

if save_to_google_drive == True and drive != None:
  uploaded = drive.CreateFile({'title': f"{output_dir}.zip"})
  uploaded.SetContentFile(f"{output_dir}.zip")
  uploaded.Upload()
  print(f"Uploaded {output_dir}.zip to Google Drive with ID {uploaded.get('id')}")